# Project Running

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gb


In [2]:
positions_table = pd.read_csv("positions.csv")
old_season_table = pd.read_csv("PlayerData21.csv")
gameweek_table = {}
agg_gameweek_table = {}
for i in range(1,39):
    gameweek_table[i] = pd.read_excel("gameweek22.xlsx", sheet_name=f"{i}")
    if (i != 1):
        agg_gameweek_table[i] = pd.read_excel("Aggregate_gameweek22.xlsx", sheet_name=f"{i}")

### Points Earned per Gameweek Function

In [3]:
#get Points function

def points_earned(starting_gw, ending_gw, priority_subs, GK_sub_index, Captain, ViceCaptain, Starting11):
    num_players = len(Captain)
    def has_played_points(gw_num_og,gw_num, playerid, subs_change):
        if (gw_num_og == 1):
            get_playerid = lambda x : old_season_table.iloc[x]['Player ID']
        else:
            get_playerid = lambda x: gameweek_table[gw_num_og].iloc[x]['Player ID']
        minutes = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['minutes'].iloc[0]
        if (minutes == 0):
            position = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['position'].iloc[0]
            if position == 'GK':
                points = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == get_playerid(int(GK_sub_index))]['total_points'].iloc[0]
                playerName = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == get_playerid(int(GK_sub_index))]['name'].iloc[0]
                playerPos = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == get_playerid(int(GK_sub_index))]['position'].iloc[0]
                print('Sub In --->' +  str(get_playerid(int(GK_sub_index))) + ' - '+ playerName + ' - ' + playerPos + ' - ' + str(points))
                return points
            else:
                if (len(subs_change) > 0):
                    while(len(subs_change)>0):
                        minutes = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == get_playerid(list(subs_change.keys())[0])]['minutes'].iloc[0]
                        if (minutes == 0):
                            subs_change.pop(list(subs_change.keys())[0])
                            continue
                        else:
                            points = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == get_playerid(list(subs_change.keys())[0])]['total_points'].iloc[0]
                            playerName = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == get_playerid(list(subs_change.keys())[0])]['name'].iloc[0]
                            playerPos = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == get_playerid(list(subs_change.keys())[0])]['position'].iloc[0]
                            print('Sub In --->' +  str(get_playerid(list(subs_change.keys())[0])) + ' - '+ playerName + ' - ' + playerPos + ' - ' + str(points))
                            subs_change.pop(list(subs_change.keys())[0])
                            return points
        else:
            playerName = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['name'].iloc[0]
            playerPos = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['position'].iloc[0]
            points = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['total_points'].iloc[0]
            print(str(playerid) + ' - '+ playerName + ' - ' + playerPos + ' - ' + str(points))
            return points
        playerName = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['name'].iloc[0]
        playerPos = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['position'].iloc[0]
        points = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid]['total_points'].iloc[0]
        print(str(playerid) + ' - '+ playerName + ' - ' + playerPos + ' - ' + str(points))
        return 0

    #Calculating number of points scored by my selected team
    def get_points(gw_num_og,gw_num):
        if (gw_num_og == 1):
            playerid = lambda x: old_season_table.iloc[x]['Player ID']
        else:
            playerid = lambda x: gameweek_table[gw_num_og].iloc[x]['Player ID'] 
        subs_change = priority_subs.copy()
        total_points = 0
        captain_played = True
        for i in range(num_players):
            if Captain[i] == 1:
                minutes = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid(i)]['minutes'].iloc[0]
                if (minutes == 0):
                    captain_played = False

        vice_played = True    
        for i in range(num_players):
            if ViceCaptain[i] == 1:
                minutes = gameweek_table[gw_num][gameweek_table[gw_num]['Player ID'] == playerid(i)]['minutes'].iloc[0]
                if (minutes == 0):
                    vice_played = False

        for i in range(num_players):
            if Starting11[i] > 0.5:
                points = has_played_points(gw_num_og,gw_num, playerid(i),subs_change)
                if (Captain[i] == 1) and captain_played:
                    total_points += 2*points
                elif (ViceCaptain[i] == 1) and not captain_played and vice_played:
                    total_points += 2*points
                else:
                    total_points += points
        return total_points

    tot = 0
    for i in range(starting_gw,ending_gw+1):
        if i != 7:
            points = get_points(starting_gw, i)
            print(f"\nGameweek-{i} Earned Points: {points}")
            print("-----------------------------------------------")
            tot += points
    print(f"\nTotal Earned in Gameweek{starting_gw} to Gameweek {ending_gw}: {tot}")
    return tot

### Part 1: Choosing the Initial team

##### Generating Lasso Regression Coefficients [based on previous season aggregate data]

In [4]:
def generate_lasso_initial():
    # lasso on columns of
    GK = old_season_table.loc[old_season_table['position'] == 'GK']
    GK_x = GK.drop(['position', 'full_name', 'Player ID', 'teamID', 'first_name', 'second_name','total_points'], axis=1)
    GK_y = GK['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01) # you can control the number of predictors through alpha
    model = ls.fit(GK_x,GK_y)
    GK_dict_lasso = dict(zip(GK_x.columns,model.coef_))

    DEF = old_season_table.loc[old_season_table['position'] == 'DEF']
    DEF_x = DEF.drop(['position', 'full_name', 'Player ID', 'teamID', 'first_name', 'second_name', 'total_points'], axis=1)
    DEF_y = DEF['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01)  # you can control the number of predictors through alpha
    model = ls.fit(DEF_x, DEF_y)
    DEF_dict_lasso = dict(zip(DEF_x.columns, model.coef_))

    MID = old_season_table.loc[old_season_table['position'] == 'MID']
    MID_x = MID.drop(['position', 'full_name', 'Player ID', 'teamID', 'first_name', 'second_name', 'total_points'], axis=1)
    MID_y = MID['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01)  # you can control the number of predictors through alpha
    model = ls.fit(MID_x, MID_y)
    MID_dict_lasso = dict(zip(MID_x.columns, model.coef_))

    FWD = old_season_table.loc[old_season_table['position'] == 'FWD']
    FWD_x = FWD.drop(['position', 'full_name', 'Player ID', 'teamID', 'first_name', 'second_name', 'total_points'], axis=1)
    FWD_y = FWD['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01)  # you can control the number of predictors through alpha
    model = ls.fit(FWD_x, FWD_y)
    FWD_dict_lasso = dict(zip(FWD_x.columns, model.coef_))
    return GK_dict_lasso, DEF_dict_lasso, MID_dict_lasso, FWD_dict_lasso



##### Generating Utility and Xp functions 

In [5]:
#Finding Indices of players in each position
goalkeeper_indices = []
defender_indices = []
midfielder_indices = []
forward_indices = []

for index, row in old_season_table.iterrows():
    if row['position'] == 'GK':
        goalkeeper_indices.append(index)
    elif row['position'] == 'DEF':
        defender_indices.append(index)
    elif row['position'] == 'MID':
        midfielder_indices.append(index)
    elif row['position'] == 'FWD':
        forward_indices.append(index)

#Utility and xP functions
def get_util(playerIndex):
    util = 0
    GK_dict_lasso, DEF_dict_lasso, MID_dict_lasso, FWD_dict_lasso = generate_lasso_initial()
    position = old_season_table.iloc[playerIndex]['position']
    if (position == 'GK'):
        for key in GK_dict_lasso.keys():
            util += GK_dict_lasso[key] * old_season_table.iloc[playerIndex][key]
    elif (position == 'DEF'):
        for key in DEF_dict_lasso.keys():
            util += DEF_dict_lasso[key] * old_season_table.iloc[playerIndex][key]
    elif (position == 'MID'):
        for key in MID_dict_lasso.keys():
            util += MID_dict_lasso[key] * old_season_table.iloc[playerIndex][key]
    elif (position == 'FWD'):
        for key in FWD_dict_lasso.keys():
            util += FWD_dict_lasso[key] * old_season_table.iloc[playerIndex][key]
    return util/50

def get_Xp(playerIndex):
    playerid = old_season_table.iloc[playerIndex]['Player ID']
    XP_gw1 = gameweek_table[1][gameweek_table[1]['Player ID'] == playerid]['xP'].iloc[0]
    return XP_gw1 


##### The model

In [52]:
def Initial_team_selection_model():
    model = gb.Model("LASSO based selection 1st team")
    model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!
    num_players = len(old_season_table)
    gameweek_number = 1
    # Defining the variables
    players = model.addVars(num_players,lb=0, vtype=gb.GRB.BINARY, name=[f"Player Name: {old_season_table.iloc[i]['full_name']}" for i in range(num_players)])
    Captain = model.addVars(num_players,lb=0,vtype=gb.GRB.BINARY, name="Captain")
    ViceCaptain = model.addVars(num_players,lb=0,vtype=gb.GRB.BINARY, name="ViceCaptain")
    Starting11 = model.addVars(num_players,lb=0, vtype=gb.GRB.BINARY, name="Starting11")
    Subs = model.addVars(num_players,lb=0, vtype=gb.GRB.INTEGER, name="Subs")
    GK_sub_index = model.addVar(lb=0,vtype=gb.GRB.INTEGER, name="GK_sub")

    #define Objective Function
    util_fun = sum(get_util(i)*(Starting11[i] + 0.4*Subs[i] + 2*Captain[i] + 1.5*ViceCaptain[i]) for i in range(num_players))
    xp_fun = sum(get_Xp(i)*(Starting11[i] + 0.4*Subs[i] + 1.5*Captain[i] + 1.4*ViceCaptain[i]) for i in range(num_players))
    model.setObjective(
        (util_fun*0.8 + xp_fun*0.2), gb.GRB.MAXIMIZE
    )

    # Defining the constraints
    # 1. Only 15 players can be selected
    model.addConstr(
        gb.quicksum(players[i] for i in range(num_players)) == 15, "15_players_selected"
    )

    # 2. Only 2 goalkeepers can be selected
    #get all goalkeeper indices
    model.addConstr(sum(players[playerid] for playerid in goalkeeper_indices) == 2, "2_goalkeepers")

    # 3. Only 5 defenders can be selected
    model.addConstr(sum(players[playerid] for playerid in defender_indices) == 5, "5_defenders")

    # 4. Only 5 midfielders can be selected
    model.addConstr(sum(players[playerid] for playerid in midfielder_indices) == 5, "5_midfielders")

    # 5. Only 3 forwards can be selected
    model.addConstr(sum(players[playerid] for playerid in forward_indices) == 3, "3_forwards")

    # 6. Squad price be a maximum of 100 million
    def getPrice(index):
        playerID = old_season_table.iloc[index]['Player ID']
        cost = gameweek_table[1].loc[gameweek_table[1]['Player ID'] == (playerID)]['cost'].iloc[0]
        return cost

    model.addConstr(
        gb.quicksum(players[i]*getPrice(i) for i in range(num_players)) <= 100, "max_squad_price"
    )

    #7. Only 3 players from a single team can be selected
    for team in old_season_table['teamID'].unique():
        model.addConstr(
            gb.quicksum(players[i] for i in range(num_players) if old_season_table.iloc[i]['teamID'] == team) <= 3, f"max_3_from_{team}"
        )

    # 8. Only 1 captain can be selected
    model.addConstr(
        gb.quicksum(Captain[i] for i in range(num_players)) == 1, "1_captain"
    )
    #Captain selected must be from a player selected to play
    model.addConstrs(Captain[i] <= players[i] for i in range(num_players))
    # captain selected from starting 11
    model.addConstrs(Captain[i] <= Starting11[i] for i in range(num_players))

    # 9. Only 1 vice captain can be selected
    model.addConstr(
        gb.quicksum(ViceCaptain[i] for i in range(num_players)) == 1, "1_vice_captain"
    )
    #Vice captain selected must be from a player selected to play
    model.addConstrs(ViceCaptain[i] <= players[i] for i in range(num_players))
    #vice captain selected from starting 11
    model.addConstrs(ViceCaptain[i] <= Starting11[i] for i in range(num_players))
    model.addConstrs(ViceCaptain[i] + Captain[i] <= 1 for i in range(num_players))

    #10. Starting 11 must be selected from the 15 players selected
    model.addConstrs(Starting11[i] <= players[i] for i in range(num_players))
    model.addConstr(sum(Starting11[i] for i in range(num_players)) == 11, "11_starting")

    #11. Subs must be selected from the 15 players selected
    model.addConstrs(Subs[i] <= players[i] for i in range(num_players))
    model.addConstr(sum(Subs[i] for i in range(num_players)) == 4, "4_subs")
    model.addConstrs(Subs[i] + Starting11[i] <= 1 for i in range(num_players))

    # 12. Min and Max number of players in starting 11 in each position
    #goalkeepers
    model.addConstr(sum(Starting11[i] for i in goalkeeper_indices) == 1, "1_goalkeeper")
    #defenders
    model.addConstr(sum(Starting11[i] for i in defender_indices) >= 3, "3_defenders")
    model.addConstr(sum(Starting11[i] for i in defender_indices) <= 5, "5_defenders")
    #midfielders
    model.addConstr(sum(Starting11[i] for i in midfielder_indices) >= 2, "2_midfielders")
    model.addConstr(sum(Starting11[i] for i in midfielder_indices) <= 5, "5_midfielders")
    #forwards
    model.addConstr(sum(Starting11[i] for i in forward_indices) >= 1, "1_forward")
    model.addConstr(sum(Starting11[i] for i in forward_indices) <= 3, "3_forwards")

    # 13. One of the subs is a goalkeeper
    model.addConstr(sum(Subs[i] for i in goalkeeper_indices) == 1, "1_goalkeeper_sub")
    model.addConstr(sum(i*Subs[i] for i in goalkeeper_indices) == GK_sub_index, "1_goalkeeper_sub")

    #########################################################################################################
    model.optimize()
    # Print the selected players in the Starting 11
    print("\nStarting 11:")
    for i in range(num_players):
        if Starting11[i].x > 0.5:
            print(old_season_table.iloc[i]['full_name'] + ' - ' + old_season_table.iloc[i]['position'])

    # Print the selected players in the Subs
    print("\nSubstitutes:")
    for i in range(num_players):
        if Subs[i].x > 0.5:
            print(old_season_table.iloc[i]['full_name'] + ' - ' + old_season_table.iloc[i]['position'])

    #print the team captain
    for i in range(num_players):
        if Captain[i].x > 0.5:
            print("\nCaptain: " + old_season_table.iloc[i]['full_name'])
    #print the vice captain
    for i in range(num_players):
        if ViceCaptain[i].x > 0.5:
            print("Vice Captain: " + old_season_table.iloc[i]['full_name'])

    #Knowing my priority subs list
    priority_subs = {}
    for i in range(num_players):
        if Subs[i].x > 0.5 and i != GK_sub_index.x:
            priority_subs[i] = get_util(i)
    priority_subs = dict(sorted(priority_subs.items(), key=lambda item: item[1], reverse=True))

    total_cost = sum(getPrice(i) for i in range(num_players) if players[i].x > 0.5)

    return [int(players[i].x) for i in range(num_players)], [int(Starting11[i].x) for i in range(num_players)], [Subs[i].x for i in range(num_players)], [int(Captain[i].x) for i in range(num_players)], [int(ViceCaptain[i].x) for i in range(num_players)], GK_sub_index.x, priority_subs, total_cost


### MidSeason

##### Generating Lasso Coeifficients [based on current season aggregate data]

In [7]:
def generate_lasso(gameweek_number):
    # lasso on columns of
    gameweek_number = gameweek_number - 1
    import warnings
    warnings.filterwarnings('ignore')
    GK = agg_gameweek_table[gameweek_number].loc[agg_gameweek_table[gameweek_number]['position'] == 'GK']
    GK_x = GK.drop(['position', 'name', 'Player ID', 'team','total_points'], axis=1)
    GK_y = GK['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01) # you can control the number of predictors through alpha
    model = ls.fit(GK_x,GK_y)
    GK_dict_lasso = dict(zip(GK_x.columns,model.coef_))

    DEF = agg_gameweek_table[gameweek_number].loc[agg_gameweek_table[gameweek_number]['position'] == 'DEF']
    DEF_x = DEF.drop(['position', 'Player ID', 'team', 'name', 'total_points'], axis=1)
    DEF_y = DEF['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01)  # you can control the number of predictors through alpha
    model = ls.fit(DEF_x, DEF_y)
    DEF_dict_lasso = dict(zip(DEF_x.columns, model.coef_))

    MID = agg_gameweek_table[gameweek_number].loc[agg_gameweek_table[gameweek_number]['position'] == 'MID']
    MID_x = MID.drop(['position', 'name', 'Player ID', 'team', 'total_points'], axis=1)
    MID_y = MID['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01)  # you can control the number of predictors through alpha
    model = ls.fit(MID_x, MID_y)
    MID_dict_lasso = dict(zip(MID_x.columns, model.coef_))

    FWD = agg_gameweek_table[gameweek_number].loc[agg_gameweek_table[gameweek_number]['position'] == 'FWD']
    FWD_x = FWD.drop(['position', 'name', 'Player ID', 'team', 'total_points'], axis=1)
    FWD_y = FWD['total_points']
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=0.01)  # you can control the number of predictors through alpha
    model = ls.fit(FWD_x, FWD_y)
    FWD_dict_lasso = dict(zip(FWD_x.columns, model.coef_))
    return GK_dict_lasso, DEF_dict_lasso, MID_dict_lasso, FWD_dict_lasso

##### Generating Utility and Xp Functions

In [8]:

def get_util_mid(playerIndex,gameweek_number):
    util = 0
    GK_dict_lasso, DEF_dict_lasso, MID_dict_lasso, FWD_dict_lasso = generate_lasso(gameweek_number)
    position = gameweek_table[gameweek_number].iloc[playerIndex]['position']
    if (position == 'GK'):
        for key in GK_dict_lasso.keys():
            util += GK_dict_lasso[key] * agg_gameweek_table[gameweek_number].iloc[playerIndex][key]
    elif (position == 'DEF'):
        for key in DEF_dict_lasso.keys():
            util += DEF_dict_lasso[key] * agg_gameweek_table[gameweek_number].iloc[playerIndex][key]
    elif (position == 'MID'):
        for key in MID_dict_lasso.keys():
            util += MID_dict_lasso[key] * agg_gameweek_table[gameweek_number].iloc[playerIndex][key]
    elif (position == 'FWD'):
        for key in FWD_dict_lasso.keys():
            util += FWD_dict_lasso[key] * agg_gameweek_table[gameweek_number].iloc[playerIndex][key]
    return util/50

def get_Xp_mid(playerIndex,gameweek_number):
    playerid = gameweek_table[gameweek_number].iloc[playerIndex]['Player ID']
    XP_gw1 = gameweek_table[gameweek_number][gameweek_table[1]['Player ID'] == playerid]['xP'].iloc[0]
    return XP_gw1 

##### The Model

In [53]:
def MidSeason_team_selection(gameweek_number, budget):
    goalkeeper_indices = []
    defender_indices = []
    midfielder_indices = []
    forward_indices = []

    for index, row in gameweek_table[gameweek_number].iterrows():
        if row['position'] == 'GK':
            goalkeeper_indices.append(index)
        elif row['position'] == 'DEF':
            defender_indices.append(index)
        elif row['position'] == 'MID':
            midfielder_indices.append(index)
        elif row['position'] == 'FWD':
            forward_indices.append(index)

    model = gb.Model("LASSO based selection 1st team")
    model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!
    num_players = len(gameweek_table[gameweek_number])
    # Defining the variables
    players = model.addVars(num_players, vtype=gb.GRB.BINARY, name=[f"Player Name: {gameweek_table[gameweek_number].iloc[i]['name']}" for i in range(num_players)])
    Captain = model.addVars(num_players,vtype=gb.GRB.BINARY, name="Captain")
    ViceCaptain = model.addVars(num_players,vtype=gb.GRB.BINARY, name="ViceCaptain")
    Starting11 = model.addVars(num_players, vtype=gb.GRB.BINARY, name="Starting11")
    Subs = model.addVars(num_players, vtype=gb.GRB.INTEGER, name="Subs")
    GK_sub_index = model.addVar(vtype=gb.GRB.INTEGER, name="GK_sub")

    #define Objective Function
    util_fun = sum(get_util_mid(i,gameweek_number)*(Starting11[i] + 0.4*Subs[i] + 2*Captain[i] + 1.5*ViceCaptain[i]) for i in range(num_players))
    xp_fun = sum(get_Xp_mid(i,gameweek_number)*(Starting11[i] + 0.4*Subs[i] + 1.4*Captain[i] + 1.2*ViceCaptain[i]) for i in range(num_players))
    model.setObjective(
        (util_fun*0.9 + xp_fun*0.1), gb.GRB.MAXIMIZE
    )

    # Defining the constraints
    # 1. Only 15 players can be selected
    model.addConstr(
        gb.quicksum(players[i] for i in range(num_players)) == 15, "15_players_selected"
    )

    # 2. Only 2 goalkeepers can be selected
    #get all goalkeeper indices
    model.addConstr(sum(players[playerid] for playerid in goalkeeper_indices) == 2, "2_goalkeepers")

    # 3. Only 5 defenders can be selected
    model.addConstr(sum(players[playerid] for playerid in defender_indices) == 5, "5_defenders")

    # 4. Only 5 midfielders can be selected
    model.addConstr(sum(players[playerid] for playerid in midfielder_indices) == 5, "5_midfielders")

    # 5. Only 3 forwards can be selected
    model.addConstr(sum(players[playerid] for playerid in forward_indices) == 3, "3_forwards")

    # 6. Squad price be a maximum of 1 million
    def getPrice(index):
        playerID = gameweek_table[gameweek_number].iloc[index]['Player ID']
        cost = gameweek_table[gameweek_number].loc[gameweek_table[gameweek_number-1]['Player ID'] == (playerID)]['cost'].iloc[0]
        return cost

    model.addConstr(
        gb.quicksum(players[i]*getPrice(i) for i in range(num_players)) <= budget, "max_squad_price"
    )

    #7. Only 3 players from a single team can be selected
    for team in gameweek_table[gameweek_number]['team'].unique():
        model.addConstr(
            gb.quicksum(players[i] for i in range(num_players) if gameweek_table[gameweek_number].iloc[i]['team'] == team) <= 3, f"max_3_from_{team}"
        )

    # 8. Only 1 captain can be selected
    model.addConstr(
        gb.quicksum(Captain[i] for i in range(num_players)) == 1, "1_captain"
    )
    #Captain selected must be from a player selected to play
    model.addConstrs(Captain[i] <= players[i] for i in range(num_players))
    # captain selected from starting 11
    model.addConstrs(Captain[i] <= Starting11[i] for i in range(num_players))

    # 9. Only 1 vice captain can be selected
    model.addConstr(
        gb.quicksum(ViceCaptain[i] for i in range(num_players)) == 1, "1_vice_captain"
    )
    #Vice captain selected must be from a player selected to play
    model.addConstrs(ViceCaptain[i] <= players[i] for i in range(num_players))
    #vice captain selected from starting 11
    model.addConstrs(ViceCaptain[i] <= Starting11[i] for i in range(num_players))
    model.addConstrs(ViceCaptain[i] + Captain[i] <= 1 for i in range(num_players))

    #10. Starting 11 must be selected from the 15 players selected
    model.addConstrs(Starting11[i] <= players[i] for i in range(num_players))
    model.addConstr(sum(Starting11[i] for i in range(num_players)) == 11, "11_starting")

    #11. Subs must be selected from the 15 players selected
    model.addConstrs(Subs[i] <= players[i] for i in range(num_players))
    model.addConstr(sum(Subs[i] for i in range(num_players)) == 4, "4_subs")
    model.addConstrs(Subs[i] + Starting11[i] <= 1 for i in range(num_players))

    # 12. Min and Max number of players in starting 11 in each position
    #goalkeepers
    model.addConstr(sum(Starting11[i] for i in goalkeeper_indices) == 1, "1_goalkeeper")
    #defenders
    model.addConstr(sum(Starting11[i] for i in defender_indices) >= 3, "3_defenders")
    model.addConstr(sum(Starting11[i] for i in defender_indices) <= 5, "5_defenders")
    #midfielders
    model.addConstr(sum(Starting11[i] for i in midfielder_indices) >= 2, "2_midfielders")
    model.addConstr(sum(Starting11[i] for i in midfielder_indices) <= 5, "5_midfielders")
    #forwards
    model.addConstr(sum(Starting11[i] for i in forward_indices) >= 1, "1_forward")
    model.addConstr(sum(Starting11[i] for i in forward_indices) <= 3, "3_forwards")

    # 13. One of the subs is a goalkeeper
    model.addConstr(sum(Subs[i] for i in goalkeeper_indices) == 1, "1_goalkeeper_sub")
    model.addConstr(sum(i*Subs[i] for i in goalkeeper_indices) == GK_sub_index, "1_goalkeeper_sub")

    #########################################################################################################
    model.optimize()
    # Print the selected players in the Starting 11
    print("\nStarting 11:")
    for i in range(num_players):
        if Starting11[i].x > 0.5:
            print(gameweek_table[gameweek_number].iloc[i]['name'] + ' - ' + gameweek_table[gameweek_number].iloc[i]['position'])

    # Print the selected players in the Subs
    print("\nSubstitutes:")
    for i in range(num_players):
        if Subs[i].x > 0.5:
            print(gameweek_table[gameweek_number].iloc[i]['name'] + ' - ' + gameweek_table[gameweek_number].iloc[i]['position'])

    #print the team captain
    for i in range(num_players):
        if Captain[i].x > 0.5:
            print("\nCaptain: " + gameweek_table[gameweek_number].iloc[i]['name'])
    #print the vice captain
    for i in range(num_players):
        if ViceCaptain[i].x > 0.5:
            print("Vice Captain: " + gameweek_table[gameweek_number].iloc[i]['name'])

    #Knowing my priority subs list
    priority_subs = {}
    for i in range(num_players):
        if Subs[i].x > 0.5 and i != GK_sub_index.x:
            priority_subs[i] = get_util_mid(i,gameweek_number)
    priority_subs = dict(sorted(priority_subs.items(), key=lambda item: item[1], reverse=True))

    total_cost = sum(getPrice(i) for i in range(num_players) if players[i].x > 0.5)

    return [int(players[i].x) for i in range(num_players)], [int(Starting11[i].x) for i in range(num_players)], [Subs[i].x for i in range(num_players)], [int(Captain[i].x) for i in range(num_players)], [int(ViceCaptain[i].x) for i in range(num_players)], GK_sub_index.x, priority_subs, total_cost


### Results

In [37]:
def calculate_budget(players, oldgw, newgw, old_cost, old_budget):
    if (oldgw == 1):
        table = old_season_table
    else:
        table = gameweek_table[oldgw]
    profits = 0
    for i in range(len(players)):
        if players[i] == 1:
            playerID = table.iloc[i]['Player ID']
            bought = gameweek_table[oldgw].loc[gameweek_table[oldgw]['Player ID'] == (playerID)]['cost'].iloc[0]
            sold = gameweek_table[newgw].loc[gameweek_table[newgw]['Player ID'] == (playerID)]['cost'].iloc[0]
            profits += round(sold,1) - round(bought,1)
    extra = old_budget-old_cost if old_budget-old_cost > 0 else 0
    tot = old_cost + profits + extra
    extra = 0
    return round(tot,1)


In [54]:
def run(splits):
    print(f"-------------------------- INITIAL TEAM : GW 1 - {splits[0]-1} --------------------------")
    players_1, Starting11_1, Subs_1, Captain_1, ViceCaptain_1, GK_sub_index_1, priority_subs_1,total_cost_1 = Initial_team_selection_model()
    print("Performance: ")
    print("--------------------------")
    initials = points_earned(1,splits[0]-1,priority_subs_1,GK_sub_index_1,Captain_1,ViceCaptain_1,Starting11_1)
    budget = calculate_budget(players_1,1,splits[0],round(total_cost_1,1),100)

    for i in range(len(splits)):
        gw_start = splits[i]
        end = splits[i+1] if i != len(splits)-1 else 38
        print(f"-------------------------- MIDSEASON TEAM : GW {gw_start} - {end} --------------------------")
        players_2, Starting11_2, Subs_2, Captain_2, ViceCaptain_2, GK_sub_index_2, priority_subs_2, total_cost_2 = MidSeason_team_selection(gw_start,budget)
        if (i != len(splits)-1):
            initials += points_earned(gw_start,splits[i+1]-1,priority_subs_2,GK_sub_index_2,Captain_2,ViceCaptain_2,Starting11_2)
            budget = calculate_budget(players_2,gw_start,splits[i+1],round(total_cost_2,1),budget)
        else:
            initials += points_earned(gw_start,38,priority_subs_2,GK_sub_index_2,Captain_2,ViceCaptain_2,Starting11_2)


    return (initials)

res = run([9,18,26,34])
# gameweek 9 -> use wildcard of first half of the season
# gameweek 18 -> use 15 free transfer points
# gameweek 26 -> use wildcard of second half of the season
# gameweek 34 -> use 15 free transfer points
print("_______________________________________________________________")
print(f"Total Points Earned in Season 22/23 = {res}")
print("_______________________________________________________________")

-------------------------- INITIAL TEAM : GW 1 - 8 --------------------------

Starting 11:
Bukayo Saka - MID
Gabriel dos Santos Magalhães - DEF
Ivan Toney - FWD
Conor Gallagher - MID
Virgil van Dijk - DEF
Mohamed Salah - MID
Trent Alexander-Arnold - DEF
João Cancelo - DEF
Ederson Santana de Moraes - GK
Bernardo Veiga de Carvalho e Silva - MID
James Ward-Prowse - MID

Substitutes:
Tyrone Mings - DEF
Armando Broja - FWD
Emmanuel Dennis - FWD
José Malheiro de Sá - GK

Captain: Mohamed Salah
Vice Captain: Trent Alexander-Arnold
Performance: 
--------------------------
13 - Bukayo Saka - MID - 6
16 - Gabriel dos Santos Magalhães - DEF - 7
80 - Ivan Toney - FWD - 8
148 - Conor Gallagher - MID - 1
280 - Virgil van Dijk - DEF - 1
283 - Mohamed Salah - MID - 12
285 - Trent Alexander-Arnold - DEF - 1
306 - João Cancelo - DEF - 7
307 - Ederson Santana de Moraes - GK - 7
311 - Bernardo Veiga de Carvalho e Silva - MID - 1
407 - James Ward-Prowse - MID - 9

Gameweek-1 Earned Points: 72
------------